In [1]:
cores="1"

import sys
sys.path.append("/path/to/PyOphidia")
sys.path.append("/path/to/esdm-pav-client")

In [ ]:
from esdm_pav_client import Workflow, Experiment, Task
experiment = Experiment(
    name="Iterative processing",
    author="CMCC",
    abstract="Perform an iterative computation over an ESDM container",
    exec_mode="sync",
    host_partition="cluster",
    ncores=cores)
t1 = experiment.newTask(name="Loop",
                type="control",
                operator='for',
                arguments={'key': 'index',
                           'counter': '1:10'})
t2 = experiment.newTask(name="Set",
                type="control",
                operator='set',
                arguments={'key': 'start|stop|previous',
                           'value': 'EVAL((&index-1)*10)|EVAL(&index*10-1)|@CUBE'},
                dependencies={t1:'cube'})
t3 = experiment.newTask(name="Wait",
                type="control",
                operator='wait',
                arguments={'type': 'file',
                           'output': 'esdm://etas.nc',
                           'measure': 'tas',
                           'subset_dims': 'time',
                           'subset_filter': '@stop',
                           'time_filter': 'no',
                           'timeout': '10'},
                dependencies={t2:''})
t4 = experiment.newTask(name="If the first loop",
                type="control",
                operator='if',
                arguments={'condition': '&index=1'},
                dependencies={t3:'input'})
t5 = experiment.newTask(name="Import operation",
                type="ophidia",
                operator='oph_importesdm2',
                arguments={'measure': 'tas',
                           'subset_dims': 'time',
                           'subset_filter': '@start_1:@stop',
                           'time_filter': 'no',
                           'operation': 'stat',
                           'args': '11'},
                dependencies={t4:'input'})
t6 = experiment.newTask(name="Else (next loops)",
                type="control",
                operator='else',
                arguments={},
                dependencies={t4:'input'})
t7 = experiment.newTask(name="Concat operation",
                type="ophidia",
                operator='oph_concatesdm2',
                arguments={'measure': 'tas',
                           'subset_dims': 'time',
                           'subset_filter': '@start_1:@stop',
                           'operation': 'stat',
                           'args': '11',
                           'cube': '@previous'},
                dependencies={t6:'input'})
t8 = experiment.newTask(name="End if",
                type="control",
                operator='endif',
                arguments={},
                dependencies={t5:'cube', t7:'cube'})
t9 = experiment.newTask(name="End loop",
                type="control",
                operator='endfor', 
                arguments={},
                dependencies={t8:'cube'})
t10 = experiment.newTask(name="Export",
                type="ophidia",
                operator="oph_exportesdm2",
                arguments={'output': 'esdm://output.nc',
                           'force': 'yes'},
                dependencies={t9:'cube'})

In [ ]:
workflow = Workflow(experiment)
workflow.submit()
workflow.monitor(frequency=1, iterative=True, visual_mode=True)